## COVID-19 US Community Mobility Trends Analysis

In this project, we will explore how the COVID-19 pandemic has impacted mobility trends in the United States, including changes in the number of visits to various types of locations (e.g. retail and recreation, grocery stores and pharmacies, parks, transit stations, workplaces, and residential areas).


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load the data

In [3]:
US_2020 = pd.read_csv('data/2020_US_Region_Mobility_Report.csv', low_memory=False)
US_2021 = pd.read_csv('data/2021_US_Region_Mobility_Report.csv', low_memory=False)
US_2022 = pd.read_csv('data/2022_US_Region_Mobility_Report.csv', low_memory=False)

Concatenate three dataframes 

In [16]:
US_Mobility = pd.concat([US_2020, US_2021, US_2022], ignore_index=True)

US_Mobility.sample(5)

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,place_id,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
66014,US,United States,Colorado,NaN,NaN,US-CO,NaN,ChIJt1YYm3QUQIcR_6eQSTGDVMc,2020-07-27,-7.0,2.0,119.0,-26.0,-37.0,8.0
324777,US,United States,Florida,Pinellas County,NaN,NaN,12103.0,ChIJmU1R49GXwogREBL8gkBLFZ8,2021-02-21,-14.0,-13.0,52.0,-27.0,-15.0,3.0
493394,US,United States,Georgia,Ben Hill County,NaN,NaN,13017.0,ChIJAbZPDA7j8YgRFv71ZBzRWdE,2022-03-27,NaN,NaN,NaN,NaN,-32.0,NaN
399312,US,United States,Alabama,Coffee County,NaN,NaN,1031.0,ChIJY8xZwLgTkogRzT0BJZXvy-A,2022-02-25,0.0,13.0,NaN,NaN,-9.0,1.0
352521,US,United States,Georgia,Habersham County,NaN,NaN,13137.0,ChIJxZH_pFFIX4gRVY9HP52Y-2A,2021-06-14,9.0,11.0,NaN,NaN,-31.0,5.0


In [10]:
US_Mobility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551057 entries, 0 to 551056
Data columns (total 15 columns):
 #   Column                                              Non-Null Count   Dtype  
---  ------                                              --------------   -----  
 0   country_region_code                                 551057 non-null  object 
 1   country_region                                      551057 non-null  object 
 2   sub_region_1                                        550083 non-null  object 
 3   sub_region_2                                        536126 non-null  object 
 4   metro_area                                          0 non-null       float64
 5   iso_3166_2_code                                     13957 non-null   object 
 6   census_fips_code                                    536424 non-null  float64
 7   place_id                                            551057 non-null  object 
 8   date                                                551055 non-n

rename columns and select relevant columns

In [23]:
# rename columns
US_Mobility = US_Mobility.rename(columns = {'sub_region_1': 'state',
                                            'sub_region_2': 'county',
                                            'retail_and_recreation_percent_change_from_baseline': 'retail_and_recreation',
                                            'grocery_and_pharmacy_percent_change_from_baseline':'grocery_and_pharmacy',
                                            'parks_percent_change_from_baseline':'parks',
                                            'transit_stations_percent_change_from_baseline':'transit_stations',
                                            'workplaces_percent_change_from_baseline':'workplaces',
                                            'residential_percent_change_from_baseline':'residential'} )

In [30]:
columns = ['state',
           'county',
           'date',
           'retail_and_recreation', 
           'grocery_and_pharmacy', 
           'parks',
           'transit_stations', 
           'workplaces', 
           'residential']
US_Mobility = US_Mobility[columns]

US_Mobility.sample(5)

,state,county,date,retail_and_recreation,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
44583,Arkansas,Saline County,2020-08-02,3.0,8.0,NaN,11.0,-12.0,2.0
521680,Georgia,Rabun County,2022-08-29,NaN,27.0,NaN,NaN,1.0,NaN
387068,Idaho,Elmore County,2021-10-02,20.0,NaN,NaN,38.0,-19.0,NaN
239984,Arizona,Santa Cruz County,2021-04-25,-28.0,-31.0,22.0,29.0,-9.0,2.0
484538,Florida,Monroe County,2022-07-08,-22.0,-10.0,-16.0,-19.0,-25.0,1.0


In [38]:
US_Mobility['year-month'] = US_Mobility['date'].str.rsplit('-', n=1, expand=True).drop(columns=1, axis=1)
US_Mobility['year-month']

0         2020-02
1         2020-02
2         2020-02
3         2020-02
4         2020-02
           ...   
551052    2022-03
551053    2022-03
551054    2022-03
551055    2022-03
551056    2022-03
Name: year-month, Length: 551057, dtype: object

### Grocery & pharmacy

Explore the mobility trends for places like grocery markets, food warehouses, farmers markets, specialty food shops, drug stores, and pharmacies.

### Parks
Explore the mobility trends for places like local parks, national parks, public beaches, marinas, dog parks, plazas, and public gardens.

### Transit stations
Explore the mobility trends for places like public transport hubs such as subway, bus, and train stations.

### Retail & recreation
Explore the mobility trends for places like restaurants, cafes, shopping centers, theme parks, museums, libraries, and movie theaters.

### Residential
Explore the mobility trends for places of residence.

### Workplaces
Explore the mobility trends for places of work.